#### Script para recortar a base de Unidades de Conservação de Proteção Integral por Estado e por Municípios

Projeto: Sistema de Apoio à Caracterização de Imóveis Rurais  
Embrapa/2023

In [2]:
# Módulos necessários
import geopandas as gpd
import os
import glob
import numpy as np

In [3]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir diretório principal
# # dirpath = input('Diretório principal: ')
# dirpath = os.getenv('INPUT_PATH', '/app/input')
# 
dirpath = os.getenv('INPUT_PATH', '/app/input')


In [4]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir caminho do shapefile arquivo original de Unidades de Conservação
# # unidades = input('Caminho Unidades de Conservação: ')
# unidades = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'unidades')
# 
unidades = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'unidades')


In [5]:
# 🚨 Substituído automaticamente
import os
# Original: # 🚨 Substituído automaticamente
# import os
# # Original: # Definir caminho do shapefile que será usado como máscara para recorte (Municípios do BR - sem Buffer)
# # limites = input('Caminho Municípios: ')
# mun_path = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'Municipios', 'BR_Municipios_2021.shp')
# 
mun_path = os.path.join(os.getenv('INPUT_PATH', '/app/input'), 'Municipios', 'BR_Municipios_2021.shp')


In [24]:
# Ler o shapefile de Unidades de Conservação
base = gpd.read_file(unidades)
base = base.to_crs(epsg=4326)
# Lendo shapefile de Municípios
lim = gpd.read_file(limites)
lim = lim.to_crs(epsg=4326)
# Criar pasta para armazenar a base de UCPI recortada por estado
uf_path = os.path.join(dirpath, 'UCPI_Estados')
os.makedirs(uf_path, exist_ok=True)
# Criar pasta para armazenar a base de UCPI recortada por municípios
mun_path = os.path.join(dirpath, 'UCPI_Municipios')
os.makedirs(mun_path, exist_ok=True)
# Selecionar apenas as feições de UCPI e sem Unidades de Conservação
ucpi = base[['UCMI_C', 'UCEI_C', 'UCFI_C', 'geometry']]
# Função para atribuir valor 0 ou 1 com base nas condições
def assign_CD_UCPI(row):
    if row['UCMI_C'] == 0 and row['UCEI_C'] == 0 and row['UCFI_C'] == 0:
        return 0
    else:
        return 1
# Aplicar a função a cada linha do DataFrame
ucpi['CD_UCPI'] = ucpi.apply(assign_CD_UCPI, axis=1)
# Excluir colunas desnecessárias
ucpi = ucpi.drop(['UCMI_C', 'UCEI_C', 'UCFI_C'], axis = 1)
# Cortar por estado
# Agrupar por estado
agrupado = lim.groupby('SIGLA_UF')
for key,values in agrupado:
    output = uf_path + fr'\UCPI_{key}.shp'
    uf = lim[lim["SIGLA_UF"] == f"{key}"]
    geodf_clip = gpd.clip(ucpi, uf, keep_geom_type=True)
    geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))
# Lista dos estados
estados = list(np.unique(lim['SIGLA_UF']))
# Cortar por município
for estado in estados:
    shapes = glob.glob(uf_path + f'**/*{estado}.shp')
    # Selecionar GeoDataFrame por estado
    select = lim[lim['SIGLA_UF'] == f'{estado}']
    # Agrupar por município
    agrupado = select.groupby('CD_MUN')
    for shape in shapes:
        for key,values in agrupado:
            ucpi = gpd.read_file(shape)
            ucpi = ucpi.to_crs(epsg=4326)
            nome_arq = os.path.basename(shape)
            nome_arq = nome_arq.replace('.shp', fr'_{key}.shp')
            output = mun_path + f"\{nome_arq}"
            mun = lim[lim["CD_MUN"] == f"{key}"]
            geodf_clip = gpd.clip(ucpi, mun, keep_geom_type=True)
            geodf_clip.to_file(driver = 'ESRI Shapefile', filename = (rf'{output}'))

c:\Users\youx\miniconda3\envs\geo\Lib\site-packages\geopandas\geodataframe.py:1538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
